In [104]:
import streamlit as st
import jsonlines
import random
import pandas as pd
import json
import fileinput
import numpy as np
from pprint import pprint

In [89]:
#Importing Dataset
olafScholzJsonLines = jsonlines.open("OlafScholz.jl")

olafScholzTwitter = pd.read_json("OlafScholz.jl", lines = True)
print("Datensatzlänge: " + str(olafScholzTwitter.info()))

print(olafScholzTwitter.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   http_status   65 non-null     int64 
 1   account_name  65 non-null     object
 2   account_data  65 non-null     object
 3   params        65 non-null     object
 4   response      65 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.7+ KB
Datensatzlänge: None


,http_status,account_name,account_data,params,response
0,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...",{'data': [{'context_annotations': [{'domain': ...
1,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...","{'data': [{'in_reply_to_user_id': '38150247', ..."
2,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...",{'data': [{'conversation_id': '144576374098134...
3,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...",{'data': [{'created_at': '2021-09-02T13:33:12....
4,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...",{'data': [{'conversation_id': '141999599421797...


In [ ]:
print("Response Metadaten: " + str(olafScholzTwitter["response"][0].keys()))
ResponseKeys = olafScholzTwitter["response"][0].keys()
print(ResponseKeys)
for Key in ResponseKeys:
    print(type(olafScholzTwitter["response"][0][Key]))
    print("{}-Metadaten: ".format(Key) + str(olafScholzTwitter["response"][0][Key]))

In [ ]:
#print(olafScholzTwitter["response"][0].len)
#print("---")
#Starting to investigate Dataset
#print(olafScholzTwitter.describe())
#print(type(olafScholzTwitter))
#print(olafScholzTwitter.keys())
#for key in olafScholzTwitter:
#    print("{} : {}".format(key, olafScholzTwitter[key]))
#    print("{}".format(key) + str(type(olafScholzTwitter[key])))
    

In [153]:
for line in olafScholzJsonLines:
    #print(line)
    keyResponse = line["response"]
    print("Response-Type:" + str(type(keyResponse)))
    print("Response-Keys:" + str(keyResponse.keys()))
    
    for key in keyResponse:
        print(type(keyResponse[key]))
        #pprint(keyResponse[key])
        #print("{}-Metadaten: ".format(key) + str(keyResponse[key].keys()))
    print("Includes-Metadaten: " + str(keyResponse["includes"].keys())+ "\n")
 
    print("Includes-Keys:" + str(keyResponse["includes"]["users"][0].keys())+"\n")
    #pprint("Example Includes:" +"\n" + str(keyResponse["includes"]["users"])+"\n")
        
    print("Meta-Metadaten: " + str(keyResponse["meta"].keys())+ "\n")
    data = keyResponse["data"]
    print("Type-Data: " + str(type(data))+ "\n")
    print(str(data[0]) + "\n")
    print("Type-Data[0]: " + str(type(data[0])) + "\n")
    print("data[0].keys(): " + str(data[0].keys()) + "\n")
    #print(dir(data[0].keys()))
    print("len(data[0].keys()): " + str(len(data[0].keys())) + "\n")
    break

Response-Type:<class 'dict'>
Response-Keys:dict_keys(['data', 'includes', 'errors', 'meta'])
<class 'list'>
<class 'dict'>
<class 'list'>
<class 'dict'>
Includes-Metadaten: dict_keys(['media', 'users', 'tweets'])

Includes-Keys:dict_keys(['location', 'id', 'public_metrics', 'username', 'profile_image_url', 'url', 'created_at', 'entities', 'description', 'name', 'protected', 'verified'])

Meta-Metadaten: dict_keys(['newest_id', 'oldest_id', 'result_count', 'next_token'])

Type-Data: <class 'list'>

{'public_metrics': {'retweet_count': 4, 'reply_count': 1, 'like_count': 5, 'quote_count': 0}, 'lang': 'de', 'possibly_sensitive': False, 'source': 'TweetDeck', 'text': 'Morgen wird Hamburg beim 8. Nationalen IT-Gipfel zum zentralen Treffpunkt der IT-Branche: http://t.co/KhmU4vxtOF http://t.co/nZclT9JM47', 'entities': {'urls': [{'start': 90, 'end': 112, 'url': 'http://t.co/KhmU4vxtOF', 'expanded_url': 'http://www.hamburg.de/pressearchiv-fhh/4388658/2014-10-08-pr-it-gipfel/', 'display_url': 'ha

In [156]:
# Importing Data into final Dataframe for ML
finalDataFrame = pd.DataFrame(columns = ["target", "ids", "date", "flag", "user", "text"])
print(finalDataFrame.head())
#dir(finalDataFrame)
for line in olafScholzJsonLines:
    #["target", "ids", "date", "flag", "user", "text"]
    newDataRow = [None, None, None, None, None, None, None]
    
    keyResponse = line["response"]
    data = keyResponse["data"]
    print(len(data))
    for tweet in data:
        #print("Tweet-ID: " + tweet["id"])
        tweetID = tweet["id"]
        authorID = tweet["author_id"]
        
        newDataRow[1] = tweetID
        print(newDataRow)
    break
    


Empty DataFrame
Columns: [target, ids, date, flag, user, text]
Index: []
100
[None, '405971418632163328', None, None, None, None, None]
[None, '405690669676503040', None, None, None, None, None]
[None, '404880325479641088', None, None, None, None, None]
[None, '404675555553189888', None, None, None, None, None]
[None, '404672568260841472', None, None, None, None, None]
[None, '404217147783938048', None, None, None, None, None]
[None, '404217060395581440', None, None, None, None, None]
[None, '403586107499352064', None, None, None, None, None]
[None, '403158422108901376', None, None, None, None, None]
[None, '402480054124310528', None, None, None, None, None]
[None, '401277842052222976', None, None, None, None, None]
[None, '401011656219574272', None, None, None, None, None]
[None, '400987221513478144', None, None, None, None, None]
[None, '400981585337393152', None, None, None, None, None]
[None, '398740028299309056', None, None, None, None, None]
[None, '398525208258437120', None, Non